In [1]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [7]:
import re #for regular expressions to allow sorting the frames

def numberRegex(e):
  #Finding all digits in the filename
  x = re.findall("(\d)", e)

  #if there are two digits in filename
  if(len(x)==2):
      return int(x[0]+""+x[1])
  #if there is one digit in the filename
  else:
      return int(x[0])


In [15]:
from scipy.io import loadmat
import pandas as pd

#Lucas directory
#C:/Users/Lucas/OneDrive - The University of Western Ontario/Documents/GitHub/SE4450-project-code/Preprocessing/Pxx-Wx-Sx_RF

#Ian directory
#H:/Documents/Github repositories/SE4450-project-code/Preprocessing/Pxx-Wx-Sx_RF

#Directory containing the RF scan data
dataFolder = 'H:/Documents/Github repositories/SE4450-project-code/Preprocessing/Pxx-Wx-Sx_RF'
vecRFDataNames = os.listdir(os.path.join(dataFolder)) #vector with the names of the images in the above folder
vecRFDataNames.sort(key=numberRegex) #Sorts the images by frame number

#Loading in the matlab file information
rawMatFile = loadmat(dataFolder+'/'+vecRFDataNames[1])

#Contains just the RF data pulled from the .mat file
RFData = [[element for element in upperElement] for upperElement in rawMatFile['I']]

#Converting python dictionary into pandas dataframe for easy manipulation (unsure if needed)
RFDataframe = pd.DataFrame(RFData)

print(df)

           0         1         2         3         4         5         6    \
0     0.003351 -0.018736  0.015842 -0.018736  0.000000 -0.017822  0.002589   
1    -0.012795 -0.014166  0.010206 -0.013252 -0.001066 -0.010358 -0.002285   
2    -0.018431  0.002285 -0.003199  0.001066  0.001676 -0.001219 -0.005484   
3    -0.010358  0.017669 -0.010815  0.012490 -0.001219  0.005331 -0.001828   
4     0.005027  0.016451 -0.010510  0.013709  0.000609  0.003960  0.000609   
...        ...       ...       ...       ...       ...       ...       ...   
2587 -0.002589  0.012490 -0.000152  0.006093 -0.000609  0.002894 -0.010053   
2588  0.001523 -0.001676  0.013100 -0.004722  0.009444 -0.004417  0.003808   
2589  0.000000  0.000000  0.004722 -0.000152  0.008225 -0.004417  0.001066   
2590  0.000305  0.001828 -0.002437  0.001371  0.003351  0.003656 -0.002285   
2591  0.000000  0.002133 -0.004417  0.004417 -0.006702  0.012034 -0.007768   

           7         8         9    ...       246       247    